In [1]:
from pyspark.sql import SparkSession
import os
os.environ['GROUP_ID'] = 'g_teg_ams_exp_ams_exp_report'
os.environ['GAIA_ID'] = '3122'
session = SparkSession.builder.config('spark.driver.memory', '8g').config('spark.executor.cores', 4).config('spark.executor.memory', '8g').getOrCreate()

Driver Log: 3bf0296d-4581-4dcb-b435-606bedb5dab9.log
Tracking URL: http://application.tdw.oa.com:8080/proxy/application_1651128848533_11742661


In [3]:
import numpy as np
import os
import pandas as pd
import math
import scipy.stats as stats
from matplotlib import pyplot as plt
expGroupId=51718
expId1=217110
expId2=217112
expId3=217114
expId4=217116
triggerValue=1
triggerId=111017984+triggerValue
beginDate='20220711'
endDate='20220717'
dateRange=[d.strftime('p_%Y%m%d') for d in pd.date_range(beginDate,endDate)]
print(dateRange)

['p_20220711', 'p_20220712', 'p_20220713', 'p_20220714', 'p_20220715', 'p_20220716', 'p_20220717']


In [5]:
from pytoolkit import TDWSQLProvider
provider = TDWSQLProvider(session, group='tl', db='teg_ams_exp_report_app')
provider.table('t_exp_trigger_bucket_d', priParts=dateRange).createOrReplaceTempView('t_exp_trigger_bucket_d')
session.sql('select * from t_exp_trigger_bucket_d limit 10').toPandas()

,partition_time,process_time,exp_id,trigger_ids,site_set,site_id,placement_type,ad_pull_mode,buying_type,bid_type,...,egmv,egmv_final,model_valid_exposure_cnt,is_cost_in_gmv_ad,cutlib_ltr_queue_len_cnt,cutlib_ltr_queue_len_sum,scoring_ltr_queue_len_cnt,scoring_ltr_queue_len_sum,docwash_ltr_queue_len_cnt,docwash_ltr_queue_len_sum
0,20220621,20220522,126252,"2883585,2949120,56754182",15,0,4,0,1,4,...,0,0,0,0,None,None,None,None,None,None
1,20220621,20220522,128574,0,21,10205334493408,0,0,1,4,...,0,0,0,0,None,None,None,None,None,None
2,20220621,20220522,128574,0,21,70008146887509,0,0,1,4,...,0,0,0,0,None,None,None,None,None,None
3,20220621,20220522,137072,0,21,10205334493408,0,0,1,1,...,0,0,0,0,None,None,None,None,None,None
4,20220621,20220522,137076,0,21,86722335951,0,0,1,4,...,0,0,0,0,None,None,None,None,None,None
5,20220621,20220522,137076,0,21,10205334493408,0,0,1,4,...,0,0,0,0,None,None,None,None,None,None
6,20220621,20220522,149066,0,21,86722335951,0,0,1,1,...,0,0,0,0,None,None,None,None,None,None
7,20220621,20220522,149126,0,21,10205334493408,0,0,1,1,...,0,0,0,0,None,None,None,None,None,None
8,20220621,20220522,149526,0,21,10205334493408,0,0,1,1,...,0,0,0,0,None,None,None,None,None,None
9,20220621,20220522,151604,0,22,82427368655,0,0,1,2,...,0,0,0,0,None,None,None,None,None,None


In [8]:
sqlString="""select bucket_id,
       sum(if(exp_id={} and array_contains(split(trigger_ids,','),'{}') and topo_width!=0,gmv/topo_width*10000,0))/1000000 as base_gmv_1,
       sum(if(exp_id={} and array_contains(split(trigger_ids,','),'{}') and topo_width!=0,gmv/topo_width*10000,0))/1000000 as base_gmv_2,
       sum(if(exp_id={} and array_contains(split(trigger_ids,','),'{}') and topo_width!=0,gmv/topo_width*10000,0))/1000000 as test_gmv_1,
       sum(if(exp_id={} and array_contains(split(trigger_ids,','),'{}') and topo_width!=0,gmv/topo_width*10000,0))/1000000 as test_gmv_2,
       sum(if(exp_id={} and topo_width!=0,gmv/topo_width*10000,0))/1000000 as base_gmv,
       sum(if(exp_id={} and array_contains(split(trigger_ids,','),'{}') and topo_width!=0,real_cost_micro_with_contract/topo_width*10000,0))/1000000 as base_cost_1,
       sum(if(exp_id={} and array_contains(split(trigger_ids,','),'{}') and topo_width!=0,real_cost_micro_with_contract/topo_width*10000,0))/1000000 as base_cost_2,
       sum(if(exp_id={} and array_contains(split(trigger_ids,','),'{}') and topo_width!=0,real_cost_micro_with_contract/topo_width*10000,0))/1000000 as test_cost_1,
       sum(if(exp_id={} and array_contains(split(trigger_ids,','),'{}') and topo_width!=0,real_cost_micro_with_contract/topo_width*10000,0))/1000000 as test_cost_2,
       sum(if(exp_id={} and topo_width!=0,real_cost_micro_with_contract/topo_width*10000,0))/1000000 as base_cost
from t_exp_trigger_bucket_d
where partition_time between {} and {}
       and process_time between {} and {}
       and exp_id in ({},{},{},{})
       and abnormal_user_type in (0)
group by bucket_id""".format(
    expId1,triggerId,
    expId2,triggerId,
    expId3,triggerId,
    expId4,triggerId,
    expId1,
    expId1,triggerId,
    expId2,triggerId,
    expId3,triggerId,
    expId4,triggerId,
    expId1,
    beginDate,endDate,
    beginDate,endDate,
    expId1,expId2,expId3,expId4
)
print(sqlString)

select bucket_id,
       sum(if(exp_id=217110 and array_contains(split(trigger_ids,','),'111017985') and topo_width!=0,gmv/topo_width*10000,0))/1000000 as base_gmv_1,
       sum(if(exp_id=217112 and array_contains(split(trigger_ids,','),'111017985') and topo_width!=0,gmv/topo_width*10000,0))/1000000 as base_gmv_2,
       sum(if(exp_id=217114 and array_contains(split(trigger_ids,','),'111017985') and topo_width!=0,gmv/topo_width*10000,0))/1000000 as test_gmv_1,
       sum(if(exp_id=217116 and array_contains(split(trigger_ids,','),'111017985') and topo_width!=0,gmv/topo_width*10000,0))/1000000 as test_gmv_2,
       sum(if(exp_id=217110 and topo_width!=0,gmv/topo_width*10000,0))/1000000 as base_gmv,
       sum(if(exp_id=217110 and array_contains(split(trigger_ids,','),'111017985') and topo_width!=0,real_cost_micro_with_contract/topo_width*10000,0))/1000000 as base_cost_1,
       sum(if(exp_id=217112 and array_contains(split(trigger_ids,','),'111017985') and topo_width!=0,real_cost_micro_w

In [9]:
df=session.sql(sqlString).toPandas()

In [10]:
df

,bucket_id,base_gmv_1,base_gmv_2,test_gmv_1,test_gmv_2,base_gmv,base_cost_1,base_cost_2,test_cost_1,test_cost_2,base_cost
0,19,4366541.500,3431644.985,5113017.167,4899979.539,8.369672e+06,4506952.32,4934240.77,4351956.93,4582598.81,12357818.37
1,0,4287221.592,4160820.388,4671903.211,4696744.338,1.009009e+07,4397453.67,4410076.47,4343884.00,4600705.49,13568755.20
2,7,4328969.637,4442551.059,4881768.308,4460182.629,9.377743e+06,3962938.19,4535093.59,4318034.52,4345969.26,12482673.07
3,6,3962574.608,3521207.726,3048900.703,5966771.393,9.746798e+06,4480770.36,4566932.42,4393901.48,4665616.77,12470440.45
4,9,2806888.493,4042769.671,3983737.035,4650834.447,1.097136e+07,4323747.57,4503286.98,4841871.54,4332792.13,12096928.79
5,17,3507246.262,3085116.991,4193649.647,3891235.324,9.104243e+06,4346259.45,4418941.27,4326376.87,4702429.05,12923373.27
6,5,3296545.701,3089481.354,5352088.864,2718818.632,7.562526e+06,4269088.81,4218425.00,4818978.55,4088712.58,12281013.58
7,1,3788461.732,3473724.726,3647215.150,2824091.750,6.908961e+06,4468027.26,4362092.35,4250389.91,4222184.20,12716796.62
8,10,3541467.172,4698605.919,3669175.473,5436040.052,1.343090e+07,4470677.04,4421761.53,4341810.11,4399882.56,12146200.56
9,3,3792378.898,4506347.339,4366671.667,4504697.539,8.884725e+06,4137032.37,4498186.73,4597439.63,4685735.66,12030436.97


In [11]:
sum(df['base_gmv_1'])

75348318.83200002

In [12]:
sum(df['base_cost_1'])

86139731.75999999

In [27]:
sum(df['base_cost'])

255457217.05000004

In [29]:
sum(df['base_gmv_1'])

75348318.83200002

In [23]:
def cal(df_in,df_out,metric):
    df1=df_in.sort_values(by=['bucket_id'])
    lift_rate_list=[]
    for i in range(0,20):
        df_temp=df1[df1.bucket_id!=i]
        test_gmv_1=df_temp['test_{}_1'.format(metric)].values
        test_gmv_2=df_temp['test_{}_2'.format(metric)].values
        base_gmv_1=df_temp['base_{}_1'.format(metric)].values
        base_gmv_2=df_temp['base_{}_2'.format(metric)].values
        base_gmv=df_temp['base_{}'.format(metric)].values
        lift=(sum(test_gmv_1)-sum(test_gmv_2))-((sum(base_gmv_1)-sum(base_gmv_2)))
        lift_rate=lift/sum(base_gmv)
        lift_rate_list.append(lift_rate)
        
    var=(20*sum(map(lambda x:x**2,lift_rate_list))-sum(lift_rate_list)**2)*19/20/20
    # print(var)
    test_gmv_1=df1['test_{}_1'.format(metric)].values
    test_gmv_2=df1['test_{}_2'.format(metric)].values
    base_gmv_1=df1['base_{}_1'.format(metric)].values
    base_gmv_2=df1['base_{}_2'.format(metric)].values
    base_gmv=df1['base_{}'.format(metric)].values
    lift=(sum(test_gmv_1)-sum(test_gmv_2))-((sum(base_gmv_1)-sum(base_gmv_2)))
    lift_rate=lift/sum(base_gmv)
    lower_conf=lift_rate-1.96*math.sqrt(var)
    upper_conf=lift_rate+1.96*math.sqrt(var)
    is_sign=1 if lower_conf>0 or upper_conf<0 else 0
    
    size=df_out.index.size
    df_out.loc[size] = [endDate,expGroupId,metric,lift,lift_rate,var,lower_conf,upper_conf,is_sign]

In [24]:
df_out=pd.DataFrame(columns=['partition_date','exp_group_id','metric','lift','lift_rate','variance','lower_conf','upper_conf','is_sign'], index=[])

In [25]:
cal(df,df_out,'gmv')
cal(df,df_out,'cost')

In [26]:
df_out

,partition_date,exp_group_id,metric,lift,lift_rate,variance,lower_conf,upper_conf,is_sign
0,20220704,51718,gmv,-1976100.348,-0.010044,0.001186,-0.077540,0.057452,0
1,20220704,51718,cost,4328793.950,0.016945,0.000046,0.003648,0.030242,1
